In [1]:
import requests
import json
import pandas as pd
from collections import defaultdict
import time
from json.decoder import JSONDecodeError
import os
import numpy as np
from typing import Dict, List, Optional, Tuple


PLAIN_TOKEN_SET = set([
    "ETH", "BTC", "BNB", "AVAX", "SOL", "ADA", "MATIC", "DOT", "TRX", "ATOM",
    "FTM", "KSM", "NEAR", "ONE", "CRO", "CELO", "XRP", "XLM", "BCH", "FIL",
    "LTC", "DOGE", "ICP", "APT", "SUI",
    "OP", "ARB", 
    "UNI", "AAVE", "COMP", "MKR", "CRV", "CVX", "BAL", "YFI", "SNX",
    "SUSHI", "RPL", "LDO", "FXS", "GNS", "PENDLE", "GMX", "LRC", "DODO",
    "MDX", "PERP", "DYDX", "RDNT", "CAKE", "RUNE", "1INCH", "BNT", "REN",
    "SPELL", "VELO",
    "USDT", "USDC", "TUSD", "BUSD", "USDP", "GUSD", 
    "USDV", "FDUSD", "EURT", "PYUSD", "XSGD",
    "XIDR", "EURC"
])

class DeFiTVRAnalyzer:
    def __init__(self, pro_api_key: str = None):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.pro_api_key = pro_api_key
        if pro_api_key:
            print(f"✅ configure DeFiLlama Pro API")
    
    def get_all_protocols(self) -> List[Dict]:
        try:
            url = "https://api.llama.fi/protocols"
            print("Obtaining all DeFi protocol data...")
            response = self.session.get(url, timeout=30)
            if response.status_code == 200:
                protocols = response.json()
                print(f"Successfully obtained {len(protocols)} Protocols")
                return protocols
            else:
                print(f"API Request failed: {response.status_code}")
                return []
        except Exception as e:
            print(f"Abnormal acquisition of protocol data: {e}")
            return []
    
    def get_protocol_token_breakdown(self, protocol_slug: str) -> Optional[Dict]:
        try:
            endpoints_to_try = []
            if self.pro_api_key:
                endpoints_to_try.extend([
                    (f"https://pro-api.llama.fi/protocol/{protocol_slug}", {'Authorization': f'Bearer {self.pro_api_key}'}),
                    (f"https://pro-api.llama.fi/protocol/{protocol_slug}/breakdown", {'Authorization': f'Bearer {self.pro_api_key}'}),
                ])
            endpoints_to_try.extend([
                (f"https://api.llama.fi/protocol/{protocol_slug}", {}),
                (f"https://api.llama.fi/tvl/{protocol_slug}", {}),
            ])
            for url, headers in endpoints_to_try:
                try:
                    response = self.session.get(url, headers=headers, timeout=30)
                    if response.status_code == 200:
                        data = response.json()
                        return data
                    elif response.status_code == 404:
                        continue
                    else:
                        print(f"API request {url} return: {response.status_code}")
                        continue
                except Exception as e:
                    print(f"request {url} abnormal: {e}")
                    continue
            print(f"All endpoints cannot be obtained {protocol_slug} data")
            return None
        except Exception as e:
            print(f"obtain {protocol_slug} token breakdown abnormal: {e}")
            return None

    def calculate_tvr(self, protocol_data: Dict, token_prices: Dict[str, float]) -> Dict:
        try:
            current_tokens = {}
            total_tvl = 0
            # method 1: currentChainTvls
            if 'currentChainTvls' in protocol_data:
                ethereum_tvl = protocol_data['currentChainTvls'].get('Ethereum', {})
                if isinstance(ethereum_tvl, dict):
                    current_tokens = ethereum_tvl
                    total_tvl = sum(v for v in ethereum_tvl.values() if isinstance(v, (int, float)))
            # method 2: tokensInUsd
            if not current_tokens and 'tokensInUsd' in protocol_data:
                tokens_data = protocol_data['tokensInUsd']
                if tokens_data and isinstance(tokens_data, list):
                    latest_tokens = tokens_data[-1] if tokens_data else {}
                    if 'tokens' in latest_tokens:
                        current_tokens = latest_tokens['tokens']
                        total_tvl = sum(v for v in current_tokens.values() if isinstance(v, (int, float)))
            # method 3: chainTvls
            if not current_tokens and 'chainTvls' in protocol_data:
                chain_tvls = protocol_data['chainTvls']
                if 'Ethereum' in chain_tvls and chain_tvls['Ethereum']:
                    latest_eth_data = chain_tvls['Ethereum'][-1] if chain_tvls['Ethereum'] else {}
                    if 'tokens' in latest_eth_data:
                        current_tokens = latest_eth_data['tokens']
                        total_tvl = sum(v for v in current_tokens.values() if isinstance(v, (int, float)))
            # method 4: tvl fallback
            if not total_tvl and 'tvl' in protocol_data:
                if isinstance(protocol_data['tvl'], list) and protocol_data['tvl']:
                    total_tvl = protocol_data['tvl'][-1].get('totalLiquidityUSD', 0)
                elif isinstance(protocol_data['tvl'], (int, float)):
                    total_tvl = protocol_data['tvl']
            plain_token_value = 0
            token_breakdown = {}
            plain_tokens_found = []
            if current_tokens:
                for token_key, token_value in current_tokens.items():
                    if isinstance(token_value, dict):
                        value = token_value.get('tvl', 0)
                        symbol = token_value.get('symbol', token_key)
                    else:
                        value = token_value
                        symbol = token_key
                    token_breakdown[symbol] = value
                    if symbol.upper() in PLAIN_TOKEN_SET:
                        plain_token_value += value
                        plain_tokens_found.append(symbol)
            tvr_ratio = (plain_token_value / total_tvl * 100) if total_tvl > 0 else 0
            return {
                'tvr_ratio': tvr_ratio,
                'plain_token_value': plain_token_value,
                'total_tvl': total_tvl,
                'plain_tokens': plain_tokens_found,
                'breakdown': token_breakdown
            }
        except Exception as e:
            print(f"Calculate TVR abnormal: {e}")
            return {
                'tvr_ratio': 0,
                'plain_token_value': 0,
                'total_tvl': 0,
                'plain_tokens': [],
                'breakdown': {}
            }

    def filter_ethereum_protocols(self, protocols: List[Dict]) -> List[Dict]:
        ethereum_protocols = []
        for protocol in protocols:
            chains = protocol.get('chains', [])
            if 'Ethereum' in chains:
                ethereum_protocols.append(protocol)
        print(f"The number of protocols on Ethereum: {len(ethereum_protocols)}")
        return ethereum_protocols

    def categorize_by_api(self, protocols: List[Dict]) -> Dict:
        target_categories = {
            "Liquid Staking",
            "Lending", 
            "Restaking",
            "Liquid Restaking",
            "Dexs",
            "CDP",
            "Onchain Capital Allocator",
            "RWA",
            "Yield"
        }
        categories = defaultdict(list)
        for protocol in protocols:
            category = protocol.get('category', 'Unknown')
            tvl = protocol.get('tvl', 0)
            if category not in target_categories:
                continue
            if tvl < 10_000_000:
                continue
            categories[category].append({
                'name': protocol.get('name'),
                'slug': protocol.get('slug'),
                'tvl': tvl,
                'chains': protocol.get('chains', []),
                'description': protocol.get('description', ''),
                'url': protocol.get('url', ''),
                'twitter': protocol.get('twitter', ''),
                'gecko_id': protocol.get('gecko_id', '')
            })
        for category in categories:
            categories[category].sort(key=lambda x: x['tvl'], reverse=True)
        return dict(categories)
        
    def get_protocols_with_tvr(self, categorized: Dict, top_n: int = 3) -> Dict:
        protocols_with_tvr = {}
        for category, protocols in categorized.items():
            print(f"\n🔍 Analyzing {category} category top {top_n} protocols...")
            category_protocols = []
            for i, protocol in enumerate(protocols[:top_n]):
                print(f" obtain {protocol['name']}  token breakdown...")
                try:
                    protocol_data = self.get_protocol_token_breakdown(protocol['slug'])
                    if protocol_data:
                        tvr_data = self.calculate_tvr(protocol_data, {})
                        enhanced_protocol = {
                            **protocol,
                            'tvr_ratio': tvr_data['tvr_ratio'],
                            'plain_token_value': tvr_data['plain_token_value'],
                            'plain_tokens': tvr_data['plain_tokens'],
                            'token_breakdown': tvr_data['breakdown']
                        }
                        category_protocols.append(enhanced_protocol)
                        print(f"    ✅ TVR: {tvr_data['tvr_ratio']:.2f}% (plain tokens) | TVL: ${tvr_data['total_tvl']/1e9:.2f}B")
                    else:
                        enhanced_protocol = {
                            **protocol,
                            'tvr_ratio': 0,
                            'plain_token_value': 0,
                            'plain_tokens': [],
                            'token_breakdown': {}
                        }
                        category_protocols.append(enhanced_protocol)
                        print(f"    ❌ Unable to obtain detailed data, use basic TVL: ${protocol['tvl']/1e9:.2f}B")
                except (JSONDecodeError, Exception) as e:
                    print(f"    ⚠️ protocol {protocol['name']}（slug={protocol['slug']}）Data pull abnormal, skipped. reason: {e}")
                    continue
                time.sleep(0.5)
            protocols_with_tvr[category] = {
                'count': len(protocols),
                'top_protocols': category_protocols
            }
        return protocols_with_tvr

    def display_results_with_tvr(self, protocols_with_tvr: Dict):
        print("\n" + "="*80)
        print("Classification and statistics of Ethereum DeFi protocol (including TVR analysis, defined by plain token)")
        print("="*80)
        sorted_categories = sorted(
            protocols_with_tvr.items(), 
            key=lambda x: x[1]['count'], 
            reverse=True
        )
        for category, data in sorted_categories:
            print(f"\n📊 {category} (total: {data['count']} protocols)")
            print("-" * 60)
            for i, protocol in enumerate(data['top_protocols'], 1):
                tvl_b = protocol['tvl'] / 1e9
                print(f"{i}. {protocol['name']}")
                print(f"   TVL: ${tvl_b:.2f}B")
                print(f"   TVR: {protocol['tvr_ratio']:.2f}% (plain tokens)")
                print(f"   plain tokens: {', '.join(protocol['plain_tokens'])}")
                print(f"   Slug: {protocol['slug']}")
                print(f"   chain: {', '.join(protocol['chains'])}")
                if protocol['token_breakdown']:
                    top_tokens = sorted(
                        protocol['token_breakdown'].items(), 
                        key=lambda x: x[1], 
                        reverse=True
                    )[:5]
                    print("   main tokens:")
                    for token, value in top_tokens:
                        print(f"     {token}: ${value/1e9:.3f}B ({value/protocol['tvl']*100:.1f}%)")
                if protocol['description']:
                    desc = protocol['description'][:100] + "..." if len(protocol['description']) > 100 else protocol['description']
                    print(f"   Description: {desc}")
                print()
    
    def save_enhanced_results(self, protocols_with_tvr: Dict, filename_prefix: str = "ethereum_defi_tvr_analysis"):
        json_filename = f"{filename_prefix}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(protocols_with_tvr, f, ensure_ascii=False, indent=2)
        print(f"✅ JSON data has been saved to: {json_filename}")
        csv_data = []
        for category, data in protocols_with_tvr.items():
            for protocol in data['top_protocols']:
                top_tokens = sorted(
                    protocol['token_breakdown'].items() if protocol['token_breakdown'] else [], 
                    key=lambda x: x[1], 
                    reverse=True
                )[:3]
                csv_data.append({
                    'Category': category,
                    'Rank_in_Category': data['top_protocols'].index(protocol) + 1,
                    'Protocol_Name': protocol['name'],
                    'Slug': protocol['slug'],
                    'TVL_USD': protocol['tvl'],
                    'TVL_Billions': protocol['tvl'] / 1e9,
                    'TVR_Percentage': protocol['tvr_ratio'],
                    'Plain_Token_Symbols': ', '.join(protocol['plain_tokens']),
                    'Plain_Token_Value_USD': protocol['plain_token_value'],
                    'Plain_Token_Value_Billions': protocol['plain_token_value'] / 1e9,
                    'Top_Token_1': top_tokens[0][0] if len(top_tokens) > 0 else '',
                    'Top_Token_1_Value_B': top_tokens[0][1] / 1e9 if len(top_tokens) > 0 else 0,
                    'Top_Token_2': top_tokens[1][0] if len(top_tokens) > 1 else '',
                    'Top_Token_2_Value_B': top_tokens[1][1] / 1e9 if len(top_tokens) > 1 else 0,
                    'Top_Token_3': top_tokens[2][0] if len(top_tokens) > 2 else '',
                    'Top_Token_3_Value_B': top_tokens[2][1] / 1e9 if len(top_tokens) > 2 else 0,
                    'Chains': ', '.join(protocol['chains']),
                    'Description': protocol['description'],
                    'URL': protocol['url'],
                    'Twitter': protocol['twitter']
                })
        df = pd.DataFrame(csv_data)
        csv_filename = f"{filename_prefix}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"✅ CSV data has been saved to: {csv_filename}")
        return json_filename, csv_filename

    def get_tvr_summary(self, protocols_with_tvr: Dict) -> pd.DataFrame:
        summary_data = []
        for category, data in protocols_with_tvr.items():
            protocols = data['top_protocols']
            if protocols:
                total_tvl = sum(p['tvl'] for p in protocols)
                avg_tvl = total_tvl / len(protocols)
                avg_tvr = sum(p['tvr_ratio'] for p in protocols) / len(protocols)
                top_protocol = protocols[0]['name']
                summary_data.append({
                    'Category': category,
                    'Total_Protocols': data['count'],
                    'Top_3_Total_TVL_B': total_tvl / 1e9,
                    'Average_TVL_B': avg_tvl / 1e9,
                    'Average_TVR_Percentage': avg_tvr,
                    'Top_Protocol': top_protocol,
                    'Top_Protocol_TVL_B': protocols[0]['tvl'] / 1e9,
                    'Top_Protocol_TVR': protocols[0]['tvr_ratio'],
                    'Top_Protocol_Plain_Tokens': ', '.join(protocols[0]['plain_tokens'])
                })
        df = pd.DataFrame(summary_data)
        df = df.sort_values('Average_TVR_Percentage', ascending=False)
        return df

    def run_complete_tvr_analysis(self, top_n: int = 3):
        print("🚀 Start DeFi protocol TVR analysis .. (as defined by plain token)")
        print(f"🎯 Goal: Calculate Top {top_n} protocol's TVR'ratio ")
        all_protocols = self.get_all_protocols()
        if not all_protocols:
            print("❌ Unable to obtain protocol data, exit the program")
            return None
        eth_protocols = self.filter_ethereum_protocols(all_protocols)
        categorized = self.categorize_by_api(eth_protocols)
        print(f"In the specified category found {len(categorized)} categories")
        protocols_with_tvr = self.get_protocols_with_tvr(categorized, top_n)
        self.display_results_with_tvr(protocols_with_tvr)
        tvr_summary = self.get_tvr_summary(protocols_with_tvr)
        print(f"\n📈 TVR analysis summary:")
        print(tvr_summary.to_string(index=False))
        json_file, csv_file = self.save_enhanced_results(protocols_with_tvr, "ethereum_defi_tvr_analysis")
        tvr_summary.to_csv('ethereum_defi_tvr_summary.csv', index=False)
        print("✅ TVR summary has been saved to: ethereum_defi_tvr_summary.csv")
        return {
            'protocols_with_tvr': protocols_with_tvr,
            'tvr_summary': tvr_summary,
            'files': {'json': json_file, 'csv': csv_file}
        }

In [2]:
if __name__ == "__main__":
    analyzer = DeFiTVRAnalyzer(pro_api_key="b9337cb9ba52a13de68d50f7f56b8734ccf8e441dfd05a8dd0ec470c70f9023d")
    results = analyzer.run_complete_tvr_analysis(top_n=5)
    if results:
        print(f"\n🎉 TVR analyze successful！")
        print(f"totally analyze {len(results['protocols_with_tvr'])} DeFi categories")
        print(f"data file has been generated: {results['files']['json']}, {results['files']['csv']}")
        all_protocols = []
        for category_data in results['protocols_with_tvr'].values():
            all_protocols.extend(category_data['top_protocols'])
        top_tvr_protocols = sorted(all_protocols, key=lambda x: x['tvr_ratio'], reverse=True)[:5]

✅ configure DeFiLlama Pro API
🚀 Start DeFi protocol TVR analysis .. (as defined by plain token)
🎯 Goal: Calculate Top 5 protocol's TVR'ratio 
Obtaining all DeFi protocol data...
Successfully obtained 6322 Protocols
The number of protocols on Ethereum: 1558
In the specified category found 9 categories

🔍 Analyzing Liquid Staking category top 5 protocols...
 obtain Lido  token breakdown...
    ✅ TVR: 0.03% (plain tokens) | TVL: $42.05B
 obtain Binance staked ETH  token breakdown...
    ✅ TVR: 6.20% (plain tokens) | TVL: $15.81B
 obtain Rocket Pool  token breakdown...
    ✅ TVR: 0.00% (plain tokens) | TVL: $3.13B
 obtain Liquid Collective  token breakdown...
    ✅ TVR: 0.00% (plain tokens) | TVL: $1.75B
 obtain StakeWise V2  token breakdown...
    ✅ TVR: 0.00% (plain tokens) | TVL: $1.54B

🔍 Analyzing Lending category top 5 protocols...
 obtain Aave V3  token breakdown...
    ✅ TVR: 6.95% (plain tokens) | TVL: $40.74B
 obtain Morpho Blue  token breakdown...
    ✅ TVR: 4.77% (plain tokens)

In [2]:
# category TVL/TVR/Fees time series 

def _as_utc(ts_like) -> pd.Timestamp:
    """Standardize any date/string/timestamp to UTC aware Timestamp."""
    ts = pd.to_datetime(ts_like)
    return ts.tz_localize("UTC") if ts.tzinfo is None else ts.tz_convert("UTC")

def _to_day(ts: int) -> pd.Timestamp:
    # DeFiLlama timestamp is in seconds; Normalize to UTC day
    return pd.to_datetime(ts, unit="s", utc=True).normalize()

def _sum_series(list_of_series: List[pd.Series]) -> pd.Series:
    if not list_of_series:
        return pd.Series(dtype=float)
    out = None
    for s in list_of_series:
        out = s if out is None else out.add(s, fill_value=0.0)
    return out

# Analyze TVL/TVR/Fees
def parse_tvl_ts_from_protocol_data(protocol_data: Dict, chain: str = "Ethereum") -> pd.Series:
    #  chainTvls[chain]['tvl']
    if isinstance(protocol_data.get("chainTvls"), dict):
        chain_blob = protocol_data["chainTvls"].get(chain)
        if isinstance(chain_blob, dict) and isinstance(chain_blob.get("tvl"), list):
            pts = chain_blob["tvl"]
            if pts:
                return pd.Series({
                    _to_day(p["date"]): float(p.get("totalLiquidityUSD", 0.0))
                    for p in pts if "date" in p
                }).sort_index()

        #  chainTvls[chain]  list
        chain_list = protocol_data["chainTvls"].get(chain)
        if isinstance(chain_list, list) and chain_list:
            rows = []
            for p in chain_list:
                if "date" not in p: 
                    continue
                dt = _to_day(p["date"])
                if isinstance(p.get("tvl"), (int, float)):
                    rows.append((dt, float(p["tvl"])))
                elif isinstance(p.get("tokens"), dict):
                    val = float(sum(v for v in p["tokens"].values()
                                    if isinstance(v, (int, float))))
                    rows.append((dt, val))
            if rows:
                return pd.Series(dict(rows)).sort_index()

    # Bottom line: Top level 'tvl' list
    if isinstance(protocol_data.get("tvl"), list) and protocol_data["tvl"]:
        pts = protocol_data["tvl"]
        return pd.Series({
            _to_day(p["date"]): float(p.get("totalLiquidityUSD", 0.0))
            if isinstance(p, dict) else np.nan
            for p in pts if isinstance(p, dict) and "date" in p
        }).sort_index()

    return pd.Series(dtype=float)

def parse_tvr_ts_from_protocol_data(protocol_data: Dict,
                                    plain_token_set: set,
                                    chain: str = "Ethereum",
                                    analyzer=None) -> Tuple[pd.Series, pd.Series, pd.Series]:
    """
    Return: plain_ts (USD), tvl_for_tvr_ts (USD, used for consistent denominator), tvr_ratio (daily ratio, reference)
    """
    def _from_tokens_points(points: List[Dict]) -> Optional[Tuple[pd.Series, pd.Series, pd.Series]]:
        rows_plain, rows_tvl = [], []
        for p in points:
            if "date" not in p:
                continue
            dt = _to_day(p["date"])
            tokens = p.get("tokens")
            if isinstance(tokens, dict) and tokens:
                plain_sum, tvl_sum = 0.0, 0.0
                for tk, val in tokens.items():
                    if isinstance(val, dict):
                        v = float(val.get("tvl", 0.0))
                        sym = (val.get("symbol") or tk or "").upper()
                    else:
                        v = float(val)
                        sym = (tk or "").upper()
                    tvl_sum += v
                    if sym in plain_token_set:
                        plain_sum += v

                rows_plain.append((dt, plain_sum))
                rows_tvl.append((dt, tvl_sum))
        if rows_plain:
            plain_s = pd.Series(dict(rows_plain)).sort_index()
            tvl_s   = pd.Series(dict(rows_tvl)).sort_index()
            tvr_s   = (plain_s / tvl_s).replace([np.inf, -np.inf], np.nan)
            return plain_s, tvl_s, tvr_s
        return None

    # 1. chainTvls[chain] - list
    if isinstance(protocol_data.get("chainTvls"), dict):
        chain_list = protocol_data["chainTvls"].get(chain)
        if isinstance(chain_list, list):
            out = _from_tokens_points(chain_list)
            if out: return out

    # 2. tokensInUsd - list
    if isinstance(protocol_data.get("tokensInUsd"), list) and protocol_data["tokensInUsd"]:
        out = _from_tokens_points(protocol_data["tokensInUsd"])
        if out: return out

    # 3.
    tvl_ts = parse_tvl_ts_from_protocol_data(protocol_data, chain=chain)
    ratio = 0.0
    if analyzer is not None and hasattr(analyzer, "calculate_tvr"):
        try:
            static = analyzer.calculate_tvr(protocol_data, token_prices={})
            ratio  = float(static.get("tvr_ratio", 0.0)) / 100.0
        except Exception:
            ratio = 0.0
    if not tvl_ts.empty and ratio >= 0:
        plain_ts = tvl_ts * ratio
        tvr_ts   = (plain_ts / tvl_ts).replace([np.inf, -np.inf], np.nan)
        return plain_ts, tvl_ts, tvr_ts

    return pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)

def fetch_fees_ts(session: requests.Session, slug: str, pro_api_key: Optional[str] = None,
                  chain: str = "Ethereum", data_type: str = "dailyFees", timeout: int = 30) -> pd.Series:
    base_urls, headers = [], {}
    if pro_api_key:
        headers = {"Authorization": f"Bearer {pro_api_key}"}
        base_urls.append("https://pro-api.llama.fi")
    base_urls.append("https://api.llama.fi")
    for base in base_urls:
        url = f"{base}/summary/fees/{slug}?dataType={data_type}"
        try:
            r = session.get(url, headers=headers, timeout=timeout)
            if r.status_code != 200:
                continue
            data = r.json()

            # breakdown: [[ts, {chain: {sub: val,...}}, ...]]
            bkd = data.get("totalDataChartBreakdown")
            if isinstance(bkd, list) and bkd:
                rows = []
                for ts, obj in bkd:
                    if not isinstance(obj, dict): 
                        continue
                    target = None
                    for k, v in obj.items():
                        if k.lower() == (chain or "").lower():
                            target = v; break
                    if isinstance(target, dict):
                        val = float(sum(v for v in target.values() if isinstance(v, (int, float))))
                        rows.append((_to_day(int(ts)), val))
                    elif isinstance(target, (int, float)):
                        rows.append((_to_day(int(ts)), float(target)))
                if rows:
                    return pd.Series(dict(rows)).sort_index()

            total = data.get("totalDataChart")
            if isinstance(total, list) and total:
                return pd.Series({_to_day(int(ts)): float(val) for ts, val in total}).sort_index()
        except Exception:
            continue
    return pd.Series(dtype=float)

# Category post-processing
def postprocess_category_df(df: pd.DataFrame,
                            category_name: str,
                            start_date: Optional[str] = None,
                            end_date: Optional[str] = None,
                            abs_thr: float = 1e6,
                            rel_ratio: float = 0.005,
                            l_consec: int = 5) -> Tuple[pd.DataFrame, Dict]:
    """
    -Time window clipping (do not fill in 0); Pre coverage period marker (before the first TVR>0)
    -valid date (>=threshold), active period (valid for 1 consecutive day)
    -Multiplier (calculated only when denominator>0); Modeling specific pairwise sequences (active periods&positive values only)
    -Coverage rate/effective coverage rate/activated sample size+inclusion marker
    """
    df = df.copy()
    df.index.name = "date"
    df = df.sort_index()

    if start_date:
        df = df[df.index >= _as_utc(start_date)]
    if end_date:
        df = df[df.index <= _as_utc(end_date)]

    df["TVR"] = df["PlainValue"]

    # Pre coverage period: all days before the first TVR>0
    first_pos = df.index[df["TVR"] > 0].min()
    df["pre_coverage"] = False
    if pd.notna(first_pos):
        df.loc[df.index < first_pos, "pre_coverage"] = True
    else:
        df["pre_coverage"] = True  # No positive TVR throughout the period

    # Covering pre coverage period
    df["TVR_ana"] = df["TVR"].where(~df["pre_coverage"])
    df["TVL_ana"] = df["TVL"].where(~df["pre_coverage"])

    # Threshold: max (absolute threshold, relative threshold=median positive TVR value of this class * rel_ratio)
    med_tvr = df.loc[df["TVR_ana"] > 0, "TVR_ana"].median()
    thr = max(abs_thr, rel_ratio * med_tvr) if pd.notna(med_tvr) else abs_thr
    df["thr"] = thr

    df["use_tvr_day"] = df["TVR_ana"].gt(0)
    df["valid_day"]   = df["use_tvr_day"] & df["TVL_ana"].ge(thr) & df["TVR_ana"].ge(thr)

    # Active period: Continuous l_consec days valid_day before entering the active period
    active_flags, consec = [], 0
    for v in df["valid_day"].astype(bool).tolist():
        consec = consec + 1 if v else 0
        active_flags.append(consec >= l_consec)
    df["active"] = active_flags

    # Daily multiplier (only when denominator>0)
    df["mult_daily"] = (df["TVL_ana"] / df["TVR_ana"]).where(df["use_tvr_day"])

    # Modeling specific pairwise sequences (active periods&positive values only)
    #TVL
    df["log_tvl_m"] = np.nan
    mask_tvl = df["active"] & df["TVL_ana"].gt(0)
    df.loc[mask_tvl, "log_tvl_m"] = np.log(df.loc[mask_tvl, "TVL_ana"])
    #TVR
    df["log_tvr_m"] = np.nan
    mask_tvr = df["active"] & df["TVR_ana"].gt(0)
    df.loc[mask_tvr, "log_tvr_m"] = np.log(df.loc[mask_tvr, "TVR_ana"])


    # Coverage/inclusion indicators
    tvr_coverage   = float(df["use_tvr_day"].mean()) if len(df) else 0.0
    valid_coverage = float(df["valid_day"].mean()) if len(df) else 0.0
    N_tvr_pos      = int(df["use_tvr_day"].sum())
    N_active       = int(df["active"].sum())

    ok_mult_cat  = (tvr_coverage >= 0.60)                       # Category multiplier
    ok_corr_tvr  = (tvr_coverage >= 0.40) and (N_tvr_pos >= 100) # TVR–Fees 
    ok_arima_tvr = (valid_coverage >= 0.60) and (N_active >= 100)# logTVR MODEL

    metrics = dict(
        category=category_name, thr=thr,
        tvr_coverage=tvr_coverage, valid_coverage=valid_coverage,
        N_tvr_pos=N_tvr_pos, N_active=N_active,
        ok_mult_cat=ok_mult_cat, ok_corr_tvr=ok_corr_tvr, ok_arima_tvr=ok_arima_tvr
    )

    # column
    cols_order = [
        "TVL","TVR","Fees","PlainValue","TVL_for_TVR","TVR ratio",
        "pre_coverage","TVL_ana","TVR_ana","thr","use_tvr_day","valid_day","active",
        "mult_daily","log_tvl_m","log_tvr_m" ]
    for c in cols_order:
        if c not in df.columns:
            df[c] = np.nan
    df = df[cols_order]
    df["category"] = category_name

    return df, metrics

# Build category data
def build_category_top5(analyzer: "DeFiTVRAnalyzer", top_n: int = 5) -> Dict[str, List[str]]:
    all_protocols = analyzer.get_all_protocols()
    eth_protocols = analyzer.filter_ethereum_protocols(all_protocols)
    categorized   = analyzer.categorize_by_api(eth_protocols)
    cat_to_slugs: Dict[str, List[str]] = {}
    for cat, items in categorized.items():
        slugs = [p["slug"] for p in items[:top_n]]
        if slugs:
            cat_to_slugs[cat] = slugs
    return cat_to_slugs

def build_category_timeseries(analyzer: "DeFiTVRAnalyzer",
                              plain_token_set: set,
                              chain: str = "Ethereum",
                              top_n: int = 5,
                              fees_data_type: str = "dailyFees",
                              sleep_sec: float = 0.3,
                              start_date: Optional[str] = "2022-04-01",
                              end_date: Optional[str] = "2025-05-01",
                              abs_thr: float = 1e6,
                              rel_ratio: float = 0.005,
                              l_consec: int = 5,
                              add_rolling: bool = False) -> Tuple[Dict[str, pd.DataFrame], pd.DataFrame, pd.DataFrame]:

    cat_to_slugs = build_category_top5(analyzer, top_n=top_n)
    if not cat_to_slugs:
        raise RuntimeError("No top five protocols were obtained for any categories.")

    out: Dict[str, pd.DataFrame] = {}
    metrics_rows: List[Dict] = []

    for cat, slugs in cat_to_slugs.items():
        tvl_series_list: List[pd.Series] = []
        fees_series_list: List[pd.Series] = []
        plain_series_list: List[pd.Series] = []
        tvl_for_tvr_series_list: List[pd.Series] = []

        for slug in slugs:
            proto_data = analyzer.get_protocol_token_breakdown(slug) or {}

            # TVL
            tvl_s = parse_tvl_ts_from_protocol_data(proto_data, chain=chain)
            if not tvl_s.empty:
                tvl_series_list.append(tvl_s)

            # TVR（plain）
            plain_s, tvl_for_tvr_s, _ = parse_tvr_ts_from_protocol_data(
                proto_data, plain_token_set, chain=chain, analyzer=analyzer
            )
            if not plain_s.empty:
                plain_series_list.append(plain_s)
            if not tvl_for_tvr_s.empty:
                tvl_for_tvr_series_list.append(tvl_for_tvr_s)

            # Fees
            fees_s = fetch_fees_ts(analyzer.session, slug,
                                   pro_api_key=analyzer.pro_api_key,
                                   chain=chain, data_type=fees_data_type)
            if not fees_s.empty:
                fees_series_list.append(fees_s)

            time.sleep(sleep_sec)

        # Sum by day 
        tvl_sum          = _sum_series(tvl_series_list)
        fees_sum         = _sum_series(fees_series_list)
        plain_sum        = _sum_series(plain_series_list)
        tvl_for_tvr_sum  = _sum_series(tvl_for_tvr_series_list)

        # Unified index, maintain NaN
        idx = tvl_sum.index.union(fees_sum.index).union(tvl_for_tvr_sum.index).union(plain_sum.index)
        tvl_sum          = tvl_sum.reindex(idx)
        fees_sum         = fees_sum.reindex(idx)
        tvl_for_tvr_sum  = tvl_for_tvr_sum.reindex(idx)
        plain_sum        = plain_sum.reindex(idx)

        tvr_ratio = (plain_sum / tvl_for_tvr_sum).replace([np.inf, -np.inf], np.nan)

        base_df = pd.DataFrame({
            "TVL": tvl_sum,
            "Fees": fees_sum,
            "PlainValue": plain_sum,      # = TVR level (USD)
            "TVL_for_TVR": tvl_for_tvr_sum,
            "TVR ratio": tvr_ratio       
        }).sort_index()

        df2, metrics = postprocess_category_df(
            base_df, category_name=cat,
            start_date=start_date, end_date=end_date,
            abs_thr=abs_thr, rel_ratio=rel_ratio, l_consec=l_consec,
        )
        out[cat] = df2
        metrics_rows.append(metrics)

    # Synthetic long table
    panel = pd.concat([df.assign(date=df.index) for df in out.values()], ignore_index=True)
    panel = panel[[
        "date","category","TVL","TVR","Fees","TVL_for_TVR","PlainValue","TVR ratio",
        "pre_coverage","TVL_ana","TVR_ana","thr","use_tvr_day","valid_day","active",
        "mult_daily","log_tvl_m","log_tvr_m"
    ]]

    # Coverage/inclusion in reports
    coverage = pd.DataFrame(metrics_rows).sort_values("tvr_coverage", ascending=False)

    return out, panel, coverage

In [4]:
# Main program
if __name__ == "__main__":
    analyzer = DeFiTVRAnalyzer(pro_api_key="b9337cb9ba52a13de68d50f7f56b8734ccf8e441dfd05a8dd0ec470c70f9023d")
    os.makedirs("_results", exist_ok=True)

    cat_ts, panel, coverage = build_category_timeseries(
        analyzer,
        plain_token_set=PLAIN_TOKEN_SET,
        chain="Ethereum",
        top_n=5,
        fees_data_type="dailyFees",
        start_date="2022-04-01",
        end_date="2025-05-01",
        abs_thr=1e6,
        rel_ratio=0.005,
        l_consec=5,
    )

    # Save: Each type of CSV+long table (contract)+coverage report
    for cat, df in cat_ts.items():
        df.to_csv(f"cat_timeseries_{cat.replace(' ', '_')}.csv")
    panel.to_csv("_results/panel_with_flags.csv", index=False)
    coverage.to_csv("_results/category_coverage_report.csv", index=False)
    print("✓ Saved per-category CSVs, panel_with_flags.csv & category_coverage_report.csv")

✅ configure DeFiLlama Pro API
Obtaining all DeFi protocol data...
Successfully obtained 6322 Protocols
The number of protocols on Ethereum: 1558
✓ Saved per-category CSVs, panel_with_flags.csv & category_coverage_report.csv
